In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from glob import glob
from tqdm import tqdm
from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def RetinaTransform(image):
    # Bringing image to 0-1 range, #ATTENTION: CHECK IF ALL IMAGES min max is 0 and 255
    image = image / 255
    
    
    image = resize(image, (64,64),anti_aliasing=False) # resizing image because original does not fit in memory.
    
    
    # Bringing data to CHW format
    # N is a batch size, C denotes a number of channels, 
    # H is a height of input planes in pixels, and W is width in pixels.
    image = image.transpose([2,0,1])
    
    #Fixing dtype to avoid runtime error and save memory
    image = torch.tensor(image ,dtype=torch.float32)
    
    return image

In [4]:
# # Load dataset into memory
# cuda_image_array = []
# data_folder = "Evaluation_Set/Validation"
# label_path = "Evaluation_Set/RFMiD_Validation_Labels.csv"
# label_frame = pd.read_csv(label_path)
# local_transform = RetinaTransform

# for image_name in tqdm(glob(data_folder+"/*")):
#     image = io.imread(image_name)

#     label_frame_index = int(image_name.split("/")[-1].split(".")[0])
#     label = label_frame[label_frame["ID"]==label_frame_index]["Disease_Risk"].values[0]

#     image = local_transform(image)
#     label =  torch.tensor(label ,dtype=torch.float32)

#     cuda_image_array.append((image,label))

100%|█████████████████████████████████████████| 640/640 [01:16<00:00,  8.36it/s]


In [79]:
class CicikNet(nn.Module):
    def __init__(self):
        super(CicikNet, self).__init__()

        
        self.lrelu = nn.LeakyReLU(0.1)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1,padding=1)

        # First fully connected layer
        self.fc1 = nn.Linear(64*64, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(p=0.9)
        
    def forward(self, x):
        x1 = torch.flatten(self.conv1(x),1)
        x = torch.cat([x1],dim=1)
        # x = self.dropout(x)
        

        x = self.fc1(x)
        x = self.lrelu(x)

        
        x = self.fc2(x)
        
        output = torch.sigmoid(x)
        return output

In [80]:
class RetinaDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_indices, transform=None):
        
        self.data = [cuda_image_array[idx] for idx in data_indices]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [81]:
# def plot_grad_flow(named_parameters):
#     ave_grads = []
#     layers = []
#     for n, p in named_parameters:
#         if(p.requires_grad) and ("bias" not in n):
#             layers.append(n)
#             ave_grads.append(p.grad.abs().mean())
#     plt.plot(ave_grads, alpha=0.3, color="b")
#     plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
#     plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
#     plt.xlim(xmin=0, xmax=len(ave_grads))
#     plt.xlabel("Layers")
#     plt.ylabel("average gradient")
#     plt.title("Gradient flow")
#     plt.grid(True)

In [82]:
def model_train_loop(net,criterion,train_dataloader):
    
    running_loss = 0.0
    predictions = []
    ground_truth = []
    
    for idx, data in enumerate(train_dataloader, 0):
        # get the inputs
        # print(i,data)
        inputs, labels = data
        inputs = inputs.to(device)
        # labels = labels.to(device)
        labels = torch.ones(1).to(device)
        
        
        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs.flatten(), labels)
        
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=10)
        
        optimizer.step()
        running_loss += loss.item()
        
        
        
        predictions.append(outputs.flatten())
        ground_truth.append(labels.flatten())
        
    # net.to('cpu')
    # plot_grad_flow(net.named_parameters())
    # plt.show()
    # net.to(device)
    
    predictions=torch.cat(predictions)
    ground_truth = torch.cat(ground_truth)
    print(predictions,ground_truth)
    healthy_f1 = 0
    # healthy_f1 = f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)
    rocauc = bauc(predictions,ground_truth)
    print('[%d, %5d] loss: %.3f, ROC: %.3f, F1_healthy: %.3f\n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))

    return running_loss, rocauc, healthy_f1

In [83]:
def model_test_loop(net, criterion, test_dataloader):
    
    with torch.no_grad():
        
        running_loss = 0.0
        test_predictions = []
        test_ground_truth = []
        
        for idx, data in enumerate(test_dataloader, 0):

            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            
            outputs = net(inputs)
            loss = criterion(outputs.flatten(), labels)
            running_loss += loss.item()

            test_predictions.append(outputs.flatten())
            test_ground_truth.append(labels.flatten())

        
        
        test_predictions=torch.cat(test_predictions)
        test_ground_truth = torch.cat(test_ground_truth)

        healthy_f1 = 0
        # healthy_f1 = f1_score((test_predictions>0.5).float().cpu(),test_ground_truth.cpu(),pos_label=0)
        rocauc = bauc(test_predictions,test_ground_truth)
        print('[%d, %5d] Validation loss: %.3f, ROC: %.3f, F1_healthy: %.3f \n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))
        
        return running_loss, rocauc,healthy_f1


In [84]:
EPOCHS = 100
criterion = nn.BCELoss()
bauc = BinaryAUROC(thresholds=None)
writer = SummaryWriter('experiments/crossval_test_experiment')

In [86]:
kf = KFold(n_splits=2)

fold_rocauc = []
fold_f1healthy = []
fold_loss = []
for idx, (train_index, test_index) in tqdm(enumerate(kf.split(cuda_image_array))):
    train_dataset = RetinaDataset(data_indices=train_index)
    test_dataset = RetinaDataset(data_indices=test_index)
    
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=0)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
    
    ciciknet = CicikNet().to(device)
    optimizer = optim.SGD(ciciknet.parameters(), lr=0.001, momentum = 0.9) # Understand why ADAM does not work with batchsize = 1
    # optimizer = optim.Adam(ciciknet.parameters(), lr=0.001)
    
    for epoch in range(EPOCHS):  # loop over the dataset multiple times
        loss_train, rocauc_train, f1healthy_train= model_train_loop(ciciknet, criterion,train_dataloader)
        loss_test, rocauc_test, f1healthy_test = model_test_loop(ciciknet, criterion,test_dataloader)
        
        writer.add_scalars('training/val rocauc', {"train_rocauc":rocauc_train,"validation_rocauc":rocauc_test}, epoch +1)
        writer.add_scalars('training/val healthy_f1', {"f1healthy_train":f1healthy_train,"f1healthy_test":f1healthy_test}, epoch +1)
        writer.add_scalars('training/val loss', {"loss_train":loss_train,"loss_test":loss_test}, epoch +1)
        
        
    loss_, rocauc, f1healthy = model_test_loop(ciciknet, criterion, test_dataloader)
    
    fold_rocauc.append(rocauc)
    fold_f1healthy.append(f1healthy)
    fold_loss.append(loss_)
    
    break

0it [00:00, ?it/s]

[1,   320] loss: 189.643, ROC: 0.496, F1_healthy: 0.000

[1,   320] Validation loss: 145.476, ROC: 0.649, F1_healthy: 0.000 

[2,   320] loss: 180.201, ROC: 0.538, F1_healthy: 0.000

[2,   320] Validation loss: 149.657, ROC: 0.634, F1_healthy: 0.000 

[3,   320] loss: 174.653, ROC: 0.586, F1_healthy: 0.000

[3,   320] Validation loss: 143.008, ROC: 0.677, F1_healthy: 0.000 

[4,   320] loss: 174.087, ROC: 0.586, F1_healthy: 0.000

[4,   320] Validation loss: 144.014, ROC: 0.660, F1_healthy: 0.000 

[5,   320] loss: 169.334, ROC: 0.637, F1_healthy: 0.000

[5,   320] Validation loss: 139.698, ROC: 0.693, F1_healthy: 0.000 

[6,   320] loss: 166.920, ROC: 0.652, F1_healthy: 0.000

[6,   320] Validation loss: 139.000, ROC: 0.691, F1_healthy: 0.000 

[7,   320] loss: 164.443, ROC: 0.664, F1_healthy: 0.000

[7,   320] Validation loss: 137.574, ROC: 0.694, F1_healthy: 0.000 

[8,   320] loss: 162.247, ROC: 0.677, F1_healthy: 0.000

[8,   320] Validation loss: 136.713, ROC: 0.695, F1_healthy: 

0it [00:20, ?it/s]

KeyboardInterrupt



In [ ]:
# EPOCHS = 100
# criterion = nn.BCELoss()
# bauc = BinaryAUROC(thresholds=None)

# kf = KFold(n_splits=5)

# fold_rocauc = []
# fold_f1healthy = []

# for idx, (train_index, test_index) in tqdm(enumerate(kf.split(cuda_image_array))):
#     train_dataset = RetinaDataset(data_indices=train_index)
#     test_dataset = RetinaDataset(data_indices=test_index)
    
#     train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=False, num_workers=0)
#     test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=0)
    
#     ciciknet = CicikNet().to(device)
#     optimizer = optim.SGD(ciciknet.parameters(), lr=0.001, momentum=0.9)
    
#     for epoch in range(EPOCHS):  # loop over the dataset multiple times
#         model_train_loop(ciciknet,train_dataloader)
    
#     rocauc, f1healthy = model_test_loop(ciciknet,test_dataloader)
    
#     fold_rocauc.append(rocauc)
#     fold_f1healthy.append(f1healthy)

In [540]:
print(fold_rocauc)

[tensor(0.8736, device='cuda:0'), tensor(0.8384, device='cuda:0'), tensor(0.7858, device='cuda:0'), tensor(0.8760, device='cuda:0'), tensor(0.8380, device='cuda:0')]


In [541]:
print(fold_f1healthy)

[0.6885245901639345, 0.5, 0.4583333333333333, 0.5818181818181819, 0.5964912280701754]
